# Introduction

OpenAssistant-Guanaco is a project focused on developing and fine-tuning chatbot models using a dataset of multilingual human-written simulated conversations. In these conversations, individuals interact with a chatbot assistant, with the twist that the responses from the assistant were authored by real people through crowdsourcing. This dataset offers a diverse range of conversational scenarios, allowing for the training and refinement of chatbot models in various languages and contexts. The primary task associated with this project involves fine-tuning chatbot models using this dataset to enhance their conversational capabilities and overall performance.

## Team Members

- Balice Matteo

- Doronzo Antonio Giuseppe

- Fabris Filip

- Masini Alessandro

# Index

2. [Analysis](#analysis)
    - [General analysis](##general-analysis)
    - [Word2Vec](##word2vec)
    - [Clustering](##clustering)

3. [Training](#training)
4. [Evaluation](#training)
5. [Conclusion](#conclusion)


### Packages

#### Install additional packages

In [ ]:
!pip install datasets
!pip install --upgrade gensim
!pip install langdetect
!pip install plotly

#### Import packages

In [ ]:
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import random, re
from gensim.models.word2vec import Word2Vec
from langdetect import detect
from pandas.core.common import flatten
from sklearn.manifold import TSNE

# Analysis

### General analysis

#### Import dataset

In [ ]:
dataset = load_dataset("timdettmers/openassistant-guanaco")

#### Analysis of dataset

In [ ]:
print("Dataset Structure:\n", dataset)
print("\nAvailable Splits:", dataset.keys())

In [ ]:
train_df = pd.DataFrame(dataset['train'], columns=['text'])
test_df = pd.DataFrame(dataset['test'], columns=['text'])

In [ ]:
df = pd.concat([train_df, test_df], axis=0)

In [ ]:
print("\nFirst few entries of the dataset:")
print(df.head())

##### Example of Human - Assistant Interaction

In [ ]:
df.iloc[0]['text'].split("###")

##### Sentence length analysis

In [ ]:
df['token_length'] = df['text'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10, 6))
sns.histplot(df['token_length'], bins=30, kde=True)
plt.title('Sentence Length Distribution')
plt.xlabel('Number of Tokens')
plt.ylabel('Frequency')
plt.show()

In [ ]:
min_length = df['token_length'].min()
max_length = df['token_length'].max()
avg_length = df['token_length'].mean()

print(f"Minimum Length: {min_length}")
print(f"Maximum Length: {max_length}")
print(f"Average Length: {avg_length}")

#### Search Engine

In [ ]:
import pyterrier as pt

if not pt.started():
    pt.init()

In [ ]:
df["docno"] = [str(i) for i in range(1, len(df) + 1)]

In [ ]:
df.info()

In [ ]:
indexer = pt.DFIndexer(
    "c:\\Users\\filip\\Desktop\\NLP-project\\local\\index", overwrite=True
)
index_ref = indexer.index(df["text"], df["docno"])
index_ref.toString()

In [ ]:
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

In [ ]:
query = "human"

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25.search(query).head(10)

In [ ]:
tf = pt.BatchRetrieve(index, wmodel="Tf")
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [ ]:
query = "human"

In [ ]:
pipeline = ((tf % 10) | (tf_idf % 10)) >> bm25

pipeline.search(query)

#### Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(df["text"])
len(vectorizer.get_feature_names_out())

In [ ]:
import nltk

# Download stop words from NLTK for the languages you are interested in.
nltk.download("stopwords")
from nltk.corpus import stopwords

english_stop_words = set(stopwords.words("english"))
spanish_stop_words = set(stopwords.words("spanish"))

# Combine the stop words sets.
combined_stop_words = english_stop_words.union(spanish_stop_words)
combined_stop_words = list(combined_stop_words)

In [ ]:
vectorizer = CountVectorizer(min_df=8, stop_words=combined_stop_words)
X = vectorizer.fit_transform(df["text"])
len(vectorizer.get_feature_names_out())

In [ ]:
print(*vectorizer.get_feature_names_out())

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Sum up the counts of each term in all documents
sums = X.sum(axis=0)  # Sum over columns to get total counts for each feature

# Connecting term names with their sums
freq = [(word, sums[0, idx]) for word, idx in zip(feature_names, range(sums.shape[1]))]
# Sorting the list of tuples by frequency
sorted_freq = sorted(freq, key=lambda x: x[1], reverse=True)

# Creating DataFrame from the sorted frequencies for easier plotting
freq_df = pd.DataFrame(sorted_freq, columns=["term", "frequency"])

# Plotting the top N most frequent terms
top_n = 20
plt.figure(figsize=(10, 8))  # Set the figure size
sns.barplot(data=freq_df.head(top_n), x="frequency", y="term", palette="viridis")
plt.title("Top 20 Most Frequent Terms")
plt.xlabel("Frequency")
plt.ylabel("Terms")
plt.show()

In [ ]:
# make another plot with the most frequent terms that most frequently appear like bubble chart, use data which you have

# Plotting the top N most frequent terms

plt.figure(figsize=(10, 8))  # Set the figure size

sns.scatterplot(data=freq_df.head(top_n), x="frequency", y="term", size="frequency", sizes=(100, 1000), palette="viridis")
plt.title("Top 20 Most Frequent Terms")
plt.xlabel("Frequency")
plt.ylabel("Terms")
plt.show()


### Word2Vec

In [ ]:
docs = dataset['train']['text'] + dataset['test']['text']

def filter_english(strings):
  english_strings = []
  for string in strings:
    try:
      if detect(string) == 'en':
        english_strings.append(string)
    except:
      print(f"Error with {string}")
  return english_strings

docs = filter_english(docs)
docs[:5]

Let's separate the sentences written by the human from the ones of the assistant:

In [ ]:
human_questions = []
assistant_answers = []

for string in docs:
  segments = string.split("### Human:")
  for segment in segments[1:]:
    #This if is added to add to the list of questions only those that have then received an answer
    if "### Assistant:" in segment:
      human_questions.append(segment.split("### Assistant:")[0].strip())

  segments = string.split("### Assistant:")
  for segment in segments[1:]:
      assistant_answers.append(segment.split("### Human:")[0].strip())

print(len(human_questions))
print(len(assistant_answers))

Let's clean the datasets and split them into different sentences:

In [ ]:
human_questions = [re.sub('[\n\t]', '', doc) for doc in human_questions]
assistant_answers = [re.sub('[\n\t]', '', doc) for doc in assistant_answers]
human_questions = [re.sub('[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '', doc) for doc in human_questions]
assistant_answers = [re.sub('[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '', doc) for doc in assistant_answers]
human_questions = [re.split('[?!.]\s', doc) for doc in human_questions]
assistant_answers = [re.split('[?!.]\s', doc) for doc in assistant_answers]

Flatten the list of lists and tokenize each word:

In [ ]:
human_questions = list(flatten(human_questions))
human_questions = [re.sub('\W', ' ', doc).lower().split() for doc in human_questions]

assistant_answers = list(flatten(assistant_answers))
assistant_answers = [re.sub('\W', ' ', doc).lower().split() for doc in assistant_answers]

#We also compute a combination of the two tet
full_text = human_questions + assistant_answers

 The minimum length of each sentence is 5, so we don't need to remove sentences that are composed by few words

 Now we train the Word2Vec on the human questions, by providing as input the tokenized words, the size of each embedding, the minimum number of occurences for each word and the context window size:



In [ ]:
model_human = Word2Vec(human_questions, vector_size=30, min_count=5, window=10)

How many words do we have in our model?

In [ ]:
len(model_human.wv)

How is each embedding vector made?

In [ ]:
term = 'house'
model_human.wv[term]

What are the most similar words to the word "short" ?

In [ ]:
term = 'short'
model_human.wv.most_similar(term)

Now, let's do the same thing but with the assistant answers:

In [ ]:
model_assistant = Word2Vec(assistant_answers, vector_size=30, min_count=5, window=10)

It's vocabulary size is more than 4 times the human one:

In [ ]:
len(model_assistant.wv)

Let's see an example of embedding:

In [ ]:
term = 'house'
model_assistant.wv[term]

And the most similar words to the word "short":

In [ ]:
term = 'short'
model_assistant.wv.most_similar(term)

Finally, let's perform the same operations with the full_text to our disposal:

In [ ]:
model_full = Word2Vec(full_text, vector_size=30, min_count=5, window=10)

The length of the vocabulary isn't much different from the one of just the assistant:

In [ ]:
len(model_full.wv)

Let's again an example of an embedding:

In [ ]:
term = 'plant'
model_full.wv[term]

And once again the most similar word to the word short:

In [ ]:
term = 'short'
model_full.wv.most_similar(term)

Has the model understood the relation between this words?

In [ ]:
vec = model_full.wv['king'] + (model_full.wv['woman'] - model_full.wv['man'])
vec

Seems it does not:

In [ ]:
model_full.wv.most_similar(vec)

And this kind of relation?

In [ ]:
vec = model_full.wv['france'] + (model_full.wv['rome'] - model_full.wv['italy'])
vec

It seems to have a better idea of what we are talking about, but still he hasn't fully understood the relation:

In [ ]:
model_full.wv.most_similar(vec)

#Visualizing the embedding vector using t-SNE

We'll now produce a graphical representation of a subset of the embeddings, because to reduce the time required for computation, we'll limit our representation to 500 random samples:

In [ ]:
random_samples_human = random.sample(list(model_human.wv.key_to_index), 500)
word_vectors_human = model_human.wv[random_samples_human]

random_samples_assistant = random.sample(list(model_assistant.wv.key_to_index), 500)
word_vectors_assistant = model_assistant.wv[random_samples_assistant]

random_samples_full = random.sample(list(model_full.wv.key_to_index), 500)
word_vectors_full = model_full.wv[random_samples_full]

We'll provide the vectors to the TSNE algorithm, to fit a model and have a 3 dimensional representation:

In [ ]:
tsne = TSNE(n_components=3, n_iter=2000)

tsne_embeddings_human = tsne.fit_transform(word_vectors_human)
tsne_embeddings_assistant = tsne.fit_transform(word_vectors_assistant)
tsne_embeddings_full = tsne.fit_transform(word_vectors_full)

We transpose the matrix, as to have each dimension in each row of the resulting matrix and we get the coordinates of each point:

In [ ]:
x_human, y_human, z_human = np.transpose(tsne_embeddings_human)
x_assistant, y_assistant, z_assistant = np.transpose(tsne_embeddings_assistant)
x_full, y_full, z_full = np.transpose(tsne_embeddings_full)

We draw the plots (we also reduced the amount of samples to be shown for a better representation):

In [ ]:
fig = px.scatter_3d(x=x_human[:150], y=y_human[:150], z=z_human[:150], text=random_samples_human[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [ ]:
fig = px.scatter_3d(x=x_assistant[:150], y=y_assistant[:150], z=z_assistant[:150], text=random_samples_assistant[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [ ]:
fig = px.scatter_3d(x=x_full[:150], y=y_full[:150], z=z_full[:150], text=random_samples_full[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

### Clustering

In [ ]:
train_df = pd.DataFrame(dataset['train'], columns=['text'])
test_df = pd.DataFrame(dataset['test'], columns=['text'])

In [ ]:
df = pd.concat([train_df, test_df], axis=0)

In [ ]:
documents = df['text'].tolist()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.8, min_df=5, stop_words='english')
vectorizer.fit(documents)
vector_documents = vectorizer.transform(documents)

In [ ]:
vocab = vectorizer.get_feature_names_out()

print(f"Length of vocabulary: {len(vocab)}")

In [ ]:
from sklearn.cluster import KMeans

# Number of clusters (languages)
k = 5

kmeans = KMeans(n_clusters=k, max_iter=100, n_init=2, verbose=True, random_state=2307)
kmeans.fit(vector_documents)
labels = kmeans.labels_

In [ ]:
for idx in range(k):
  centroid = kmeans.cluster_centers_[idx]

  # Sort terms according to their weights 
  # (argsort goes from lowest to highest, we reverse the order through slicing)
  sorted_terms = centroid.argsort()[::-1]

  # Print out the top 10 terms for the cluster
  print("Cluster " + str(idx) + ":")
  print([vocab[j] for j in sorted_terms[:20]])

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(2)
reduced_data = svd.fit_transform(vector_documents)

[x,y] = np.transpose(reduced_data)
fig = plt.figure(figsize=(15, 10))

plt.xscale('log')
plt.yscale('log')
sns.scatterplot(x=x, y=y, hue=labels, palette=['red', 'blue', 'green', 'purple', 'orange'])
plt.legend(title="Cluster")

# Training